### Transform peptide distance data to interactions (1 or 0)
* Transform the distances between the Calpha atoms in the peptides to 1 or 0 values depending on the interaction cutoff.
* This will be used for further aggregation analyses

In [ ]:
import MDAnalysis as mda
import MDAnalysis.analysis.distances
from MDAnalysis import transformations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import utils_mda.seq_manipulation as seq_manipulation
from importlib import reload
reload(seq_manipulation)

from MDAnalysis.analysis import leaflet
plt.style.use('default')

In [3]:
def get_combinations(u, peptide, pep_num, pairs=None):
    """
    Get list of residue residue combinations

    Input: 
        peptide: list of strin
            list of peptides
        pepnum: int 
            number of peptides of each type
        pairs: 
    """
    _, new_dict = seq_manipulation.get_aa_sequence(u, peptide, pep_num)
    peptide_ranges = seq_manipulation.get_peptide_range(peptide, list(new_dict.keys())[0], pep_num)
    comb_list = []
    for i in peptide_ranges.keys():
        for j in peptide_ranges.keys():
            comb_list.extend([(x, y) for x in peptide_ranges[i] for y in peptide_ranges[j] if (y, x) not in comb_list])
    return list(set(comb_list))

In [20]:
for peptide_type in ["WF1a_only", "WF2_only", "WF1a_WF2"]:
    peptide_name = peptide_type.split('_')[0] if "only" in peptide_type else peptide_type
    filen = f"../data/distances_new/distance_over_time_{peptide_name}.csv"
    print(filen)
    pd_concat = pd.read_csv(filen)
    pd_concat = pd_concat.drop(columns=["Unnamed: 0"])
    pd_concat

    pep_dict = {1: (1, 20),
                2: (21, 40),
                3: (41, 60),
                4: (61, 80),
                5: (81, 105),
                6: (106, 130),
                7: (131, 155),
                8: (156, 180)}
    if peptide_type == "WF1a_only":
        pep_dict = {1: (1, 20),
                2: (21, 40),
                3: (41, 60),
                4: (61, 80),
                5: (81, 100),
                6: (101,120),
                7: (121, 140),
                8: (141, 160),
                }
    if peptide_type == "WF2_only":
        pep_dict = {1: (1, 25),
                2: (26, 50),
                3: (51, 75),
                4: (76, 100),
                5: (101, 125),
                6: (126, 150),
                7: (151, 175),
                8: (176, 200)}
    def map_res_to_pep(res):
        for k, v in pep_dict.items():
            if res in range(v[0], v[1]+1):
                return f"pep{k}"


    pd_concat["Peptide1"] = pd_concat.apply(lambda x: map_res_to_pep(x["Peptide1"]), axis=1)
    pd_concat["Peptide2"] = pd_concat.apply(lambda x: map_res_to_pep(x["Peptide2"]), axis=1)

    combinations_list = ["pep1_pep2", "pep1_pep3", "pep1_pep4", "pep1_pep5", "pep1_pep6", "pep1_pep7", "pep1_pep8",
                        "pep2_pep3", "pep2_pep4", "pep2_pep5", "pep2_pep6", "pep2_pep7", "pep2_pep8",
                        "pep3_pep4", "pep3_pep5", "pep3_pep6", "pep3_pep7", "pep3_pep8",
                        "pep4_pep5", "pep4_pep6", "pep4_pep7", "pep4_pep8",
                        "pep5_pep6", "pep5_pep7", "pep5_pep8",
                        "pep6_pep7", "pep6_pep8",
                        "pep7_pep8"]
    df_interactions2 = pd_concat.copy()
    df_interactions2["Peptide_Pair"] = df_interactions2["Peptide1"] + "_" + df_interactions2["Peptide2"]
    df_interactions2 = df_interactions2[df_interactions2["Peptide_Pair"].isin(combinations_list)]
    df_interactions2 = df_interactions2.drop(columns=["Peptide1", "Peptide2"])
    df_interactions2 = df_interactions2.groupby(["Peptide_Pair", "Time(ns)"])["mindist"].min().unstack(-1)
    df_interactions2 = df_interactions2.astype('float')

    peptide_name = peptide_type

    
    for cutoff in [6]:
        df_interactions_to_save = df_interactions2.mask(df_interactions2>cutoff, 0)
        df_interactions_to_save = df_interactions_to_save.mask(df_interactions_to_save>0, 1)
        file_to_write = f"../data/interactions_data_new/{peptide_type}_interactions_true_false_100timestep_cutoff{cutoff}.csv"
        df_interactions_to_save.to_csv(file_to_write)
        print("Writing file ", file_to_write)


/Users/mirunaserian/Documents/PhD_finalyear/WF1a_WF2_synergy/data/distances_new/distance_over_time_WF1a.csv
Writing file  ../data/interactions_data_new/WF1a_only_interactions_true_false_100timestep_cutoff5.5.csv
/Users/mirunaserian/Documents/PhD_finalyear/WF1a_WF2_synergy/data/distances_new/distance_over_time_WF2.csv
Writing file  ../data/interactions_data_new/WF2_only_interactions_true_false_100timestep_cutoff5.5.csv
/Users/mirunaserian/Documents/PhD_finalyear/WF1a_WF2_synergy/data/distances_new/distance_over_time_WF1a_WF2.csv
Writing file  ../data/interactions_data_new/WF1a_WF2_interactions_true_false_100timestep_cutoff5.5.csv
